In [99]:
import pandas as pd
import numpy as np
import helper_function as hf
import warnings
warnings.filterwarnings('ignore')
%run helper_function.py 

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)

In [100]:
final_df, noc = prepare_olympic_dataset("athlete_events.csv", "noc_regions.csv")

In [101]:
sport_dict = {'Rugby': True,
       'Alpinism':False, 'Speed Skating':False, 'Ice Hockey':True, 'Nordic Combined':False,
       'Rhythmic Gymnastics':False, 'Short Track Speed Skating':False, 'Baseball':True,
       'Softball':True, 'Tug-Of-War':True, 'Ski Jumping':False, 'Lacrosse':True, 'Curling':True,
       'Military Ski Patrol':True, 'Cricket':True, 'Croquet':False, 'Motorboating':True,
       'Basque Pelota':False, 'Aeronautics':False, 'Jeu De Paume':False, 'Racquets':False,
       'Roque':False, 'Athletics':False, 'Hockey':True, 'Football':True, 'Wrestling': False, 'Boxing':False, 'Judo':False,
       'Taekwondo':False, 'Shooting':False, 'Weightlifting':False, 'Swimming':True, 'Cycling':False,
       'Alpine Skiing':False, 'Gymnastics':False, 'Fencing':False, 'Handball':True, 'Tennis':True,
       'Volleyball':True, 'Rowing':True, 'Table Tennis':True, 'Trampolining':False,
       'Cross Country Skiing':False, 'Badminton':False, 'Sailing':True, 'Bobsleigh':True,
       'Archery':False, 'Canoeing':False, 'Snowboarding':False, 'Biathlon':False, 'Basketball':True,
       'Beach Volleyball':True, 'Figure Skating':False, 'Polo':True, 'Equestrianism':True,
       'Water Polo':True, 'Art Competitions':False, 'Modern Pentathlon':False, 'Diving':False,
       'Luge':False, 'Freestyle Skiing':False, 'Triathlon':False, 'Skeleton':False,
       'Synchronized Swimming':True, 'Golf':False, 'Rugby Sevens':True
    
}
final_df = correct_team_medals_won(final_df, sport_dict)


In [102]:
polity_dff2 = prepare_polity_dataset("p5v2018.xls", noc)


In [103]:
polity_dff3 = get_polityshift_column(polity_dff2)

In [104]:
mapper = dict(zip(final_df.NOC, final_df.region))

In [105]:
country_dict = {'BOSNIA': 'BIH', 'CONGO-BRAZZAVILLE': 'CGO', 'CONGO BRAZZAVILLE': 'CGO',
                    'VIETNAM NORTH': 'VIE', 'TIMOR LESTE': 'TLS', 'GERMANY WEST': 'FRG', "COTE D'IVOIRE": 'CIV',
                    'KOREA SOUTH': 'KOR', 'SOUTH VIETNAM': 'VIE', 'SUDAN-NORTH': 'SUD',
                    'TRINIDAD AND TOBAGO': 'TTO', 'UNITED KINGDOM': 'GBR', 'USSR': 'URS', 'YUGOSLAVIA': 'YUG',
                    'SERBIA AND MONTENEGRO': 'YUG', 'YEMEN SOUTH': 'YMD', 'CONGO KINSHASA': 'COD'}
polity_dff3 = map_polity_region_dataset(country_dict, polity_dff2, mapper)


In [106]:
plot_country_medal_polity(final_df, polity_dff3, 'FRANCE', 1929, 2014)

In [107]:
plot_country_medal_to_participants_ratio(final_df, polity_dff3, 'FRANCE', 1929, 2014)

In [108]:
plot_country_age_polity(final_df, polity_dff3, 'FRANCE', 1929, 2014)

In [109]:
country_male_female_ratio(final_df, polity_dff3, 'FRANCE', 1929, 2014)

In [110]:
plot_country_season_wise_participants(final_df, polity_dff3, 'INDIA', 1929, 2014)